In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [3]:
train_trs = pd.read_csv('train_transaction.csv')

In [3]:
pd.set_option('display.max_columns', None)
train_trs_num_descr = train_trs.describe()
# из describe удаляем TransactionID, isFraud, а также категориальные колонки
train_trs_num_descr = train_trs_num_descr.drop(columns=['TransactionID', 'isFraud', 'card1', 'card2', 'card3', 'card5', 'addr1', 'addr2'])
print(train_trs_num_descr)
pd.reset_option('display.max_columns')

       TransactionDT  TransactionAmt          dist1         dist2  \
count   5.905400e+05   590540.000000  238269.000000  37627.000000   
mean    7.372311e+06      135.027176     118.502180    231.855423   
std     4.617224e+06      239.162522     371.872026    529.053494   
min     8.640000e+04        0.251000       0.000000      0.000000   
25%     3.027058e+06       43.321000       3.000000      7.000000   
50%     7.306528e+06       68.769000       8.000000     37.000000   
75%     1.124662e+07      125.000000      24.000000    206.000000   
max     1.581113e+07    31937.391000   10286.000000  11623.000000   

                  C1             C2             C3             C4  \
count  590540.000000  590540.000000  590540.000000  590540.000000   
mean       14.092458      15.269734       0.005644       4.092185   
std       133.569018     154.668899       0.150536      68.848459   
min         0.000000       0.000000       0.000000       0.000000   
25%         1.000000       1.0000

In [4]:
# рассчитываем count, unique, top, freq для категориальных колонок с числовыми значениями
# добавляем в describe категориальные колонки, у которых числовые значения
def add_categorical_int_inplace(df, df_describe, name):
    counts = df.groupby(name)[name].count()
    top_elem = counts.idxmax()
    top_elem_freq = counts[top_elem]
    s = df[name]
    df_describe.loc[:, name] = [s.count(), len(counts), top_elem, top_elem_freq]


In [5]:
train_trs_categor_descr = train_trs.describe(include='object')
for name in ['card1', 'card2', 'card3', 'card5', 'addr1', 'addr2']:
    add_categorical_int_inplace(train_trs, train_trs_categor_descr, name)

print(train_trs_categor_descr)

       ProductCD   card4   card6 P_emaildomain R_emaildomain      M1      M2  \
count     590540  588963  588969        496084        137291  319440  319440   
unique         5       4       4            59            60       2       2   
top            W    visa   debit     gmail.com     gmail.com       T       T   
freq      439670  384767  439938        228355         57147  319415  285468   

            M3      M4      M5      M6      M7      M8      M9   card1  \
count   319440  309096  240058  421180  244275  244288  244288  590540   
unique       2       3       2       2       2       2       2   13553   
top          T      M0       F       F       F       F       T    7919   
freq    251731  196405  132491  227856  211374  155251  205656   14932   

           card2     card3     card5     addr1     addr2  
count   581607.0  588975.0  586281.0  524834.0  524834.0  
unique     500.0     114.0     119.0     332.0      74.0  
top        321.0     150.0     226.0     299.0     

In [6]:
train_idn = pd.read_csv('train_identity.csv')

In [7]:
# describe для числовых признаков для таблицы train_identity
train_idn_num_descr = train_idn.loc[:,'id_01':'id_11'].describe()
print(train_idn_num_descr)

In [8]:
pd.set_option('display.max_columns', None)
# describe для категориальных признаков для таблицы train_identity
train_idn_categor_descr = train_idn.loc[:,'id_12':].describe(include='object')
for name in ['id_13', 'id_14', 'id_17', 'id_18', 'id_19', 'id_20', 'id_21', 'id_22', 'id_24', 'id_25', 'id_26', 'id_32']:
    add_categorical_int_inplace(train_idn, train_idn_categor_descr, name)
    
print(train_idn_categor_descr)

pd.reset_option('display.max_columns')

           id_12   id_15   id_16                 id_23  id_27   id_28   id_29  \
count     144233  140985  129340                  5169   5169  140978  140978   
unique         2       3       2                     3      2       2       2   
top     NotFound   Found   Found  IP_PROXY:TRANSPARENT  Found   Found   Found   
freq      123025   67728   66324                  3489   5155   76232   74926   

             id_30        id_31      id_33           id_34   id_35   id_36  \
count        77565       140282      73289           77805  140985  140985   
unique          75          130        260               4       2       2   
top     Windows 10  chrome 63.0  1920x1080  match_status:2       T       F   
freq         21155        22000      16874           60011   77814  134066   

         id_37   id_38 DeviceType DeviceInfo     id_13    id_14     id_17  \
count   140985  140985     140810     118666  127320.0  80044.0  139369.0   
unique       2       2          2       1786     

In [25]:
# меняем NaN на число -999 в числовых колонках в таблицах train_transaction и train_identity
num_filler = -999

def replace_nan_in_columns(df, col_names, filler):
    length = len(df)
    for name in col_names:
        for i in range(length):
            if np.isnan(df.loc[i,name]):
                df.loc[i,name] = filler


replace_nan_in_columns(train_trs, train_trs.loc[:,['TransactionDT', 'TransactionAmt', 'dist1', 'dist2']].columns, num_filler)
replace_nan_in_columns(train_trs, train_trs.loc[:,'D1':'D15'].columns, num_filler)
replace_nan_in_columns(train_trs, train_trs.loc[:,'V1':].columns, num_filler)

replace_nan_in_columns(train_idn, train_idn.loc[:,'id_01':'id_11'].columns, num_filler)

NameError: name 'train_idn' is not defined

In [27]:
# train_trs.to_csv('train_transaction_with_filler.csv')